In [2]:
import matplotlib.pyplot as plt 
import pylab as plb 
from math import *
import math
from math import floor
import numpy as np
import pandas as pd
#import pymc3 as pm
#import theano.tensor as tt
from scipy import stats
from scipy import optimize
from scipy import integrate
from scipy.integrate import trapz
from scipy.integrate import quad
from random import random, randint
import math as math
from sympy import symbols, solve, Eq
from sympy import *
import types
from sympy.core.add import Add
from matplotlib import pyplot

In [3]:
#DEFINING PROBABILITY FUNCTIONS
#For z
epsilon=0.0001
kap=2*(1/sqrt(epsilon)-1+sqrt(1-epsilon))
kap1=2*(1/sqrt(epsilon)-1)

def phi(x):
    return kap/sqrt(x)

#define the function phi
a = 0.01
b = 0.99
n = 11
y = np.linspace(a, b, n)
f=y*(1-y+y**2)**(5/2)/(y*(1-y))**3/2
I_trapz = trapz(f,y)
print(I_trapz)


#define probability density
def D0(x0):
    return 1/(2*np.pi*0.1**2)*exp(-(1-x0)**2/(2*0.1**2))

def D0bar(x0):
    return 1/(2*np.pi*0.1**2)*exp(-(1-x0)**2/(2*0.1**2))
#define the integral of the initial condition
v=quad(D0,0,1)
intD0=v[0]
##
w=quad(D0bar,0,1)
intD0bar=w[0]
#defining the weights

def v(tau1, tau2, z, x):
    f=  1
    return f**(5/2)/(sqrt(z)+(1-z)**3/2)*exp(3.57066164/sqrt(x)*(tau1-tau2))

def s(tau, taun, x):
    return exp(3.57066164/sqrt(x)*(tau-taun))


24684.74761807137


In [ ]:
#going up to taumax instead of tau

#STEP 1

#defining some variables
tau0=0.0001
taumax=0.02
Nstep=0.0005
#R=np.random.uniform(0.5,1.0)
#R=0.99
xsol= np.array([])
w=np.array([])
#z = [0 for a in range(10000)]
#taustr=[0 for a in range(10000)]
#x = [0 for a in range(10000)]
events = 0
#R=random()

def D_a(x):
    return taumax/(sqrt(x)*(1-x)**(3/2))*exp(-np.pi*taumax**2/(1-x))


#for tau in np.arange(tau0,taumax,Nstep):
for j in range(1,1001):
    R=random()
    wnum=1
    x0 = 1
    z = [0 for a in range(10000)]
    taustr=[0 for a in range(10000)]
    x = [0 for a in range(10000)]

    #Calculation of tau1
    if (math.isclose(R, 0.0) == False):# & (math.isclose(phi(x0), 0.0) == False):
        tau1=tau0-np.log(R)/(phi(x0))
        taustr[0]=tau0
        taustr[1]=tau1
        x[0]=x0
        
        #wprime = np.array([])
        
        if tau1>taumax:
            xsol = np.append(xsol, [x0]) #store the solution for the first step
            w=np.append(w, s(taumax, taustr[0], x0)) #store the weight for the first step
            events = events+1
            print(j, '\t', taustr[1])
            continue
            #STEP 2+  
        else:
            i=0
            while (taumax>=taustr[i+1]):
                i=i+1
                #if  i>5000:
                  #break
                R1=random()
                R2=random()
                
                #Calculation of the z value according to the paper
                if R1<=kap1/kap:
                    znum=float(1-epsilon/(R2+(1-R2)*sqrt(epsilon))**2)
                if R1>kap1/kap:
                    znum=float(R2**2*(1-epsilon))
              
                z[i]= znum
                x[i]=x[i-1]*z[i] #calculation of the x
                if (x[i]<1e-4):
                  break
                wnum=wnum*v(taustr[i], taustr[i-1], z[i],x[i-1]) #number that calculates the weight
                    
                    #Calculation of taun
                    #if (math.isclose(R, 0.0) == False)& (math.isclose(phi(x[i-1]), 0.0) == False):
                taustr[i+1]=taustr[i]-np.log(R)/phi(x[i])
                    #taustr[i+1]=taustr[i+1]*s(tau, taustr[i+1], x[i])
                #wprime[i] = wnum #array of the weight 
                if (x[i]>=1e-4) & (taustr[i+1]>taumax) & (x[i]<0.99):# & (np.isnan(x[i]) == False):
                  xsol = np.append(xsol, x[i]) #store x values in array 
                  #wprime = np.append(wprime, wnum) #array of the weight 
                  #w=np.append(w, wprime[i])
                  w=np.append(w, wnum*s(taumax, taustr[i], x[i]))
                  events = events+1
                  print(j, '\t', taustr[i+1], '\t' , x[i], '\t', wnum*s(taumax, taustr[i], x[i]))
                  break 


bins = 50                            
w = w/(events*(1/1000))

x_analytical = np.linspace(0.0001, 0.99, 100)
D_analytical = np.array([])
for i in x_analytical:
  D_analytical =np.append(D_analytical, D_a(i))
print(D_analytical)

               

#Plotting the histogram of the evolution equation

fig,ax=plt.subplots(figsize=(12,9))
n, bins2, patches = plt.hist(xsol, bins, weights=w)
with open('gluon_static_values_0.02tau.txt', 'w') as file1:
    content = ['D(x,tau)', '\t', 'x', '\n']
    file1.writelines(content)
    for i in range(0, len(n)):
      print(bins2[i], '\t', n[i], file = file1)
      #file1.writelines(content_i)
    file1.close()
#(counts, bins) = np.histogram(xsol)
#plt.hist(bins[:-1], bins, weights=counts)
#plt.hist(xsol, bins=500,weights=w, histtype='step')
plt.hist(xsol, bins=bins,weights=w, color='blue', label='numerical solution')
plt.plot(x_analytical, D_analytical, color= 'red', label ='analytical solution')
pyplot.yscale('log')
pyplot.ylim(1e-4, 10e3)
pyplot.ylabel('D(x,t)')
pyplot.xlabel('x')
pyplot.title('Gluon evolution in QGP for static medium')
pyplot.legend()
ax.set_xlim([0,1.1])
textstr = '\n'.join((
    r'$\tau_{0}=%.4f$' % (tau0, ),
    r'$\tau_{max}=%.3f$' % (taumax, )))

pyplot.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top')

plt.grid()
pyplot.yscale('log')
pyplot.savefig('gluon_static_0.8tau.jpg', dpi=None, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format='jpeg', transparent=False, bbox_inches=None, pad_inches=0.1, frameon=None, metadata=None)